# LSTM Model Training

### Import Libraries, Download Data, Stock Data

In [1]:
import sys
!"{sys.executable}" -m pip uninstall -y numpy
!"{sys.executable}" -m pip install numpy
!{sys.executable} -m pip install pandas yfinance scikit-learn tensorflow matplotlib

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.1 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 252.5 MB 1.5 MB/s eta 0:00:0112███████▍                      | 74.5 MB 7.0 MB/s eta 0:00:26
     |████████████████████████████████| 7.8 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 1.9 MB/s eta 0:00:0

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

/Users/elizabethlarez/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Matplotlib is building the font cache; this may take a moment.


In [3]:
ticker = "AAPL"
data = yf.download(ticker, start="2010-01-01", end="2023-11-13")
df = data[['Close']].reset_index()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


### Preprocess Data and Create Sequences

In [4]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

### Split and Reshape Data

In [5]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

### Build and Train Model

In [6]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, verbose=1)

Epoch 1/100


/Users/elizabethlarez/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0017 - val_loss: 0.0012
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 3.2589e-05 - val_loss: 7.7399e-04
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 3.2475e-05 - val_loss: 8.7390e-04
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.0844e-05 - val_loss: 6.6293e-04
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 2.9256e-05 - val_loss: 0.0015
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 3.3971e-05 - val_loss: 6.8659e-04
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.5383e-05 - val_loss: 6.1588e-04
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.0619e-05 - val_loss: 0.0013
Epoch 9/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.3539e-05 - val_loss: 7.0235e-04
Epoch 10/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.0841e-05 - val_loss: 9.7239e-04
Epoch 11/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.3749e-05 - val_loss: 5.2

### Make Predictions and RMSE

In [7]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
rmse = np.sqrt(np.mean((predictions - y_test)**2))
print(f"LSTM RMSE: {rmse}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
LSTM RMSE: 3.6201784970557007
